<a href="https://colab.research.google.com/github/linus3003/PropVisionDashboard/blob/master/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q xlrd
!git clone https://github.com/linus3003/AI-talents.git
!ls AI-talents
%cd AI-talents

Cloning into 'AI-talents'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 78 (delta 42), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (78/78), done.
alldata0301.csv     alldata_final.csv	  linksandtext.csv	 links.csv
alldata.csv	    date_translation.csv  linksandtextfinal.csv  links_new.csv
alldata_final2.csv  linksandtext_2.csv	  linksandtext_new.csv	 README.md
/content/AI-talents


In [ ]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/linus3003/AI-talents
   7a53129..e38f986  main       -> origin/main
Updating 7a53129..e38f986
error: Your local changes to the following files would be overwritten by merge:
	alldata.csv
Please commit your changes or stash them before you merge.
Aborting


In [ ]:
!pip install unidecode
import unidecode
import pandas as pd
import regex as re
import numpy as np
import dateutil.parser as dp
from datetime import date
import time
import sys


     |████████████████████████████████| 245kB 4.1MB/s 


In [ ]:
scraped_df = pd.read_csv('linksandtextfinal.csv', usecols=["url", "text"])


In [ ]:
class_df = scraped_df
class_df.drop_duplicates(subset = ['url'], inplace=True)
class_df.dropna(subset=['url','text'],inplace=True)
class_df.reset_index(drop=True,inplace=True)

In [ ]:
class_df['cleantext'] = [unidecode.unidecode(row) for row in class_df.text]
class_df['cleantext'] = [re.sub('\/', ' ', row) for row in class_df.cleantext]
class_df['cleantext'] = [row.strip() for row in class_df.cleantext]
class_df['cleantext'] = [re.sub(' +', ' ', row) for row in class_df.cleantext]

In [ ]:
nbk_main = re.compile("Bezugsfertig (([\S]+[\s]*){1,4}) Einheiten")
nbk_2 = re.compile("Bezugsfertig ([a-zA-Z0-9]* [[a-zA-Z0-9]*) ")
nbk_3 = re.compile("Bezugsfertig (.*) Einheiten")
nbk_4 = re.compile("Bezugsfertig.*[\s]*([0-9]{4})")
hwg_1 = re.compile("Fertigstellung (([\S]*[\s]*){0,9}20[0-3][0-9]) Architekten")
hwg_year = re.compile("Fertigstellung (.*20[0-3][0-9])")
wr_state = re.compile("Stand des Bauvorhabens: ([a-zA-Z0-9]* [[a-zA-Z0-9]*)")

In [ ]:
class_df['scraped_class'] = [nbk_main.search(str(x)).group(1) if nbk_main.search(str(x)) is not None else
                             
                             nbk_2.search(str(x)).group(1) if nbk_2.search(str(x)) is not None else
                             nbk_3.search(str(x)).group(1) if nbk_3.search(str(x)) is not None else
                             nbk_4.search(str(x)).group(1) if nbk_4.search(str(x)) is not None else
                             hwg_1.search(str(x)).group(1) if hwg_1.search(str(x)) is not None else
                             hwg_year.search(str(x)).group(1) if hwg_year.search(str(x)) is not None else
                             wr_state.search(str(x)).group(1) if wr_state.search(str(x)) is not None else ""
                             for x in class_df.cleantext]

In [ ]:
vsl = re.compile("[vV][\S]*s[\S]*")
ca = re.compile("[cC][\S]*a[\S]*")
wsdot = re.compile("[\s\.]+")
ab = re.compile ("[aA][b]")


In [ ]:
class_df['scraped_class']= [re.sub(vsl, "", row) for row in class_df['scraped_class']]
class_df['scraped_class']= [re.sub(ca, "", row) for row in class_df['scraped_class']]
class_df['scraped_class']= [re.sub(wsdot, " ", row) for row in class_df['scraped_class']]
class_df['scraped_class']= [re.sub(ab, " ", row) for row in class_df['scraped_class']]

In [ ]:
#dev_status feature
for index,row in class_df.iterrows():
  

  if row.scraped_class == "" and re.search("Bezugsfertig.* (.*) ", row.cleantext):
    row['scraped_class']= re.search("Bezugsfertig.* (.*) ", row.cleantext).group(0)

  if re.search("[bB]au",row.scraped_class):
    row['dev_status'] = "Bau gestartet"
  
  elif re.search("[pP]rojektiert", row.cleantext):
    row['dev_status'] = 'geplant'

  else:

    try: 
 
      string = ['Fertigstellung', str(dp.parse(row['scraped_class'], fuzzy=True).year)]
      row['dev_status'] = " ".join(string)
  
    except dp.ParserError:

      if re.search("([0-9]{4})", row.scraped_class):
        row['dev_status'] = "Fertigstellung "+str(re.search("([0-9]{4})", row.scraped_class).group(0))

      elif re.search("[sS]ofort",row.scraped_class) or re.search("[bB]ezugsfrei",row.scraped_class):
        
        row['dev_status'] = "Fertiggestellt"
      elif re.search("[aA]nfrage", row.scraped_class):
      
        row['dev_status'] = "auf Anfrage"
      else:
  
        row['dev_status'] = "unbekannt"

    except TypeError:

      if re.search("([0-9]{4})", row.scraped_class):
        row['dev_status'] = "Fertigstellung "+str(re.search("([0-9]{4})", row.scraped_class).group(0))

      elif re.search("[sS]ofort",row.scraped_class) or re.search("[bB]ezugsfrei",row.scraped_class):
        
        row['dev_status'] = "Fertiggestellt"
      elif re.search("[aA]nfrage", row.scraped_class):
      
        row['dev_status'] = "auf Anfrage"
      else:
      
        row['dev_status'] = "unbekannt"



  class_df.at[index, 'dev_status'] = row.dev_status

 
    


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
strasse = re.compile("([aA-zZ\-]+[\s]*[sS]tr[aA-zZ\.]*[\s][0-9]+[aA-zZ]*[\s]*[\-\,\&\+]*[\s]*[0-9]*[aA-zZ]*,[\s][0-9]{5}[\s][aA-zZ]*) ")

In [ ]:
eigenn = re.compile("([aA-zZ\-]+[\s][0-9]+[aA-zZ]*[\s]*[\-\,\&\+]*[\s]*[0-9]*[aA-zZ]*,[\s][0-9]{5} [aA-zZ]*) ")

In [ ]:
noplzstr = re.compile("([aA-zZ\-]+[\s]*[sS]tr[\S]*[\s]*[0-9]+[aA-zZ]*[\s]*[\-\,\&\+]*[\s]*[0-9]*[aA-zZ]*,[\s][\S])")

In [ ]:
rest = re.compile("([aA-zZ\-]+[\s]*[aA-zZ\-]*[\s]*[0-9]*[aA-zZ]*[\s]*[\-\,\&\+]*[\s]*[0-9]*[aA-zZ]*,[\s][0-9]*[\s]*[aA-zZ\-]+)")

In [ ]:
class_df['address_sc'] = [strasse.search(str(x)).group(1) if strasse.search(str(x)) is not None else                          
                          eigenn.search(str(x)).group(1) if eigenn.search(str(x)) is not None else
                          noplzstr.search(str(x)).group(1) if noplzstr.search(str(x)) is not None else 
                          rest.search(str(x)).group(1) if rest.search(str(x)) is not None
                          else None
                          for x in class_df['cleantext']]

In [ ]:
sub = re.compile("[aA-zZ]*[\s]*[\-\&\+][\s]*[0-9]+[aA-zZ]*")

In [ ]:
class_df['address_clean']= [re.sub(sub,"", row) if row is not None else None for row in class_df['address_sc']]

In [ ]:
class_df['is_unit']= ['True' if "wohneinheit" in str(url) else 'False' for url in class_df.url]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#GEOCODING Nominatim

!pip install geopandas
!pip install geopy
import geopy
from geopy.extra.rate_limiter import RateLimiter

     |████████████████████████████████| 972kB 6.6MB/s 
     |████████████████████████████████| 6.5MB 18.9MB/s 
     |████████████████████████████████| 14.8MB 302kB/s 


In [ ]:
locator = geopy.geocoders.Nominatim(user_agent="4heroes")

# 1 - convenient function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)
# 2- - create location column
class_df['location'] = class_df['address_clean'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
class_df['point'] = class_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
class_df.to_csv('/content/drive/MyDrive/alldata.csv')




Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
RateLimiter caught an error, retrying (0/2 tries). Called with (*('A-H, 40 A-K',), **{}).
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/geopy/geocoders/base.py", line 344, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/usr/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/usr/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.6/urllib/request.py", line 1368, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/lib/python3.6/urllib/request.py", line 1328, in do_open
    r = h.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1373, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 31

In [ ]:
alldata = pd.read_csv('alldata.csv')

In [ ]:
alldata = alldata.dropna(subset=['point']).copy()
alldata.reset_index(drop=True, inplace=True)

In [ ]:
alldata[['latitude', 'longitude', 'altitude']] = alldata['point'].str.split(',', n=2, expand=True)

In [ ]:
alldata['latitude'] = [re.sub("\(","",row) for row in alldata['latitude']]

In [ ]:
alldata.to_csv('alldata.csv')

In [ ]:
#load data 
alldata = pd.read_csv('alldata.csv', index_col=0)

In [ ]:
alldata.drop(['altitude', 'scraped_class', 'address_sc', 'text'], axis=1, inplace=True)

In [ ]:
alldata[:5]

,url,text,cleantext,scraped_class,dev_status,address_sc,address_clean,is_unit,location,point,latitude,longitude,altitude
0,https://www.neubaukompass.de/neubau/54-east-be...,Denkmalschutz Provisionsfrei Grünberger Straße...,Denkmalschutz Provisionsfrei Grunberger Strass...,sofort,Fertiggestellt,"Gubener Strasse 11-13d, 10243 Berlin","Gubener Strasse 11, 10243 Berlin",False,"11, Gubener Straße, Friedrichshain, Friedrichs...","(52.5143853, 13.4463976, 0.0)",52.5143853,13.4463976,0.0)
1,https://www.neubaukompass.de/neubau/hu150-berlin/,"Hultschiner Damm 150, 12623 Berlin / Mahlsdorf...","Hultschiner Damm 150, 12623 Berlin Mahlsdorf B...",Ende 2021,Fertigstellung 2021,"Damm 150, 12623 Berlin","Damm 150, 12623 Berlin",False,"150, Hultschiner Damm, Mahlsdorf Süd, Mahlsdor...","(52.4882279, 13.6059149, 0.0)",52.4882279,13.6059149,0.0)
2,https://www.neubaukompass.de/neubau/sophienpar...,"Freienwalder Straße 19A, 13359 Berlin / Weddin...","Freienwalder Strasse 19A, 13359 Berlin Wedding...",Q4 2022,Fertigstellung 2022,"Freienwalder Strasse 19A, 13359 Berlin","Freienwalder Strasse 19A, 13359 Berlin",False,"19A, Freienwalder Straße, Soldiner Kiez, Gesun...","(52.558338, 13.392609, 0.0)",52.558338,13.392609,0.0)
3,https://www.neubaukompass.de/neubau/vert-wohne...,"Johannisstraße 14 - 19, 10117 Berlin / Mitte B...","Johannisstrasse 14 - 19, 10117 Berlin Mitte Be...",Q2 2023,Fertigstellung 2023,"Johannisstrasse 14 - 19, 10117 Berlin","Johannisstrasse 14, 10117 Berlin",False,"14, Johannisstraße, Spandauer Vorstadt, Mitte,...","(52.5244441, 13.3908804, 0.0)",52.5244441,13.3908804,0.0)
4,https://www.neubaukompass.de/neubau/vert-wohne...,VERT - Wohnen AM TACHELES VERT - Wohnen AM TAC...,VERT - Wohnen AM TACHELES VERT - Wohnen AM TAC...,2023,Fertigstellung 2023,"Johannisstrasse 14 - 19, 10117 Berlin","Johannisstrasse 14, 10117 Berlin",True,"14, Johannisstraße, Spandauer Vorstadt, Mitte,...","(52.5244441, 13.3908804, 0.0)",52.5244441,13.3908804,0.0)


In [ ]:
alldata.to_csv('alldata.csv')

In [ ]:
alldata = pd.read_csv('alldata.csv', index_col=0)

In [ ]:
#POI data from Overpass API

!pip install overpy
import overpy

     |████████████████████████████████| 51kB 3.9MB/s 
  Created wheel for overpy: filename=overpy-0.4-cp36-none-any.whl size=45772 sha256=4e2c1591a302a7e1050f0a0432eb0f0b2c2e71d03f5a1a6e607c71533fa42c7b
  Stored in directory: /root/.cache/pip/wheels/10/53/c2/e6b6f97e7bb419193bd3aafbe38628666f0f93a1cec9dc521d
Successfully built overpy


In [ ]:
api = overpy.Overpass()

#schools5km= api.query("node[amenity=school](around:5000,"+str(lat)+","+str(long)+");out;")
#unis10km = api.query("node[amenity=university](around:10000,"+str(lat)+","+str(long)+");out;")
#bars5km = api.query("node[amenity=bar](around:5000,"+str(lat)+","+str(long)+");out;")
#rest5km = api.query("node[amenity=restaurant](around:5000,"+str(lat)+","+str(long)+");out;")

#office = api.query("node[office=yes](around:5000,"+str(lat)+","+str(long)+");out;")

In [ ]:
alldata['schools5km']=0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
total = len(alldata)

In [ ]:
def schools_5km(x):

  try:
        s5k= api.query("node[amenity=school](around:5000,"+str(x[9])+","+str(x[10])+");out;")
        sys.stdout.write('\r'+ str(x[0])+"/"+ str(total)+ " ["+ str(round((x[0]/total)*100,1))+ "%] completed - "+ str(len(s5k.nodes)) + " inserted")
        return len(s5k.nodes)

  except:
        print(", Exception at index "+ str(x[0])+" ")
        time.sleep(20)
        return np.NaN
  
  #return x['schools5km']


In [ ]:
s5k= api.query("node[amenity=school](around:5000,"+str(lat)+","+str(long)+");out;")

In [ ]:
alldata['schools5km']=alldata.apply(schools_5km, axis=1)
alldata.to_csv('alldata_overpass.csv')
!cp alldata_overpass.csv /content/drive/MyDrive


450/5206 [8.6%] completed - + 16 insertedException at index 451 
3866/5206 [74.3%] completed - + 124 insertedException at index 3867 
5205/5206 [100.0%] completed - + 30 inserted

In [ ]:
alldata.cleantext.loc[10]

'Schulzendorfer Strasse 111, 12526 Berlin Bohnsdorf Wohntyp Eigentumswohnung Mikroapartment Preis 138.700 EUR - 322.510 EUR Zimmeranzahl 1 - 2 Zimmer Wohnflache 23 m2 - 46 m2 Bezugsfertig 28.02.2022 Einheiten 25 Kategorie Gehoben'

In [ ]:
alldata.loc[13]

In [ ]:
allprice = re.compile("([\d]+[\.]*[\d]*[\.]*[\d]*[\s]EUR[\s]*[\-]*[\s]*([\d]+[\.]*[\d]*[\.]*[\d]*[\s]EUR)*)")
price= re.compile("([\d]+[\.]*[\d]*[\.]*[\d]*)[\s]EUR[\s]*[\-]*[\s]*([\d]+[\.]*[\d]*[\.]*[\d]*)*")
allrooms = re.compile("([\d]+[\s]*+[\-]*[\s]*(bis)*[\s]*[\d]*[\s]*[\-]*(und mehr )*Zimmer)")
allrooms2 = re.compile("Zimmeranzahl ([0-9]+)")
allsqft = re.compile("([\d]+([\,][\d]+)*[\s]*m2[\s]*[\-]*(bis)*[\s]*([\d]+([\,][\d]+)*[\s]*m2)*)")
rooms = re.compile("([\d]+)[\D]*([\d]+)*")
sqft = re.compile("([\d]+[\,]*[\d]*)[\s]*m2[\s]*[\-]*b*i*s*[\s]*([\d]+[\,]*[\d]*)*")

In [ ]:
alldata['allprice'] = [allprice.search(str(x)).group(1) if allprice.search(str(x)) is not None else ""
                    for x in alldata.cleantext]

In [ ]:
pd.set_option('display.max_colwidth', -1)
alldata[['allprice','cleantext']].loc[1000:1010]

In [ ]:
alldata['allrooms'] = [allrooms.search(str(x)).group(1) if allrooms.search(str(x)) is not None else 
                       allrooms2.search(str(x)).group(1) if allrooms2.search(str(x)) is not None else ""
                      for x in alldata.cleantext]

In [ ]:
alldata['allsqft'] = [allsqft.search(str(x)).group(1) if allsqft.search(str(x)) is not None else ""
                    for x in alldata.cleantext]

In [ ]:
alldata2 = alldata.copy()

In [ ]:
alldata.allrooms.unique()

array(['1 - 2 Zimmer', '5 Zimmer', '1 - 4 Zimmer', '2 - 5 Zimmer', '',
       '2 - 4 Zimmer', '4 - 6 und mehr Zimmer', '3 - 5 Zimmer',
       '1 - 6 und mehr Zimmer', '4 - 5 Zimmer', '1 - 3 Zimmer',
       '4 Zimmer', '1 - 5 Zimmer', '3 Zimmer', '3 - 4 Zimmer',
       '2 - 6 Zimmer', '3 - 6 Zimmer', '2 Zimmer', '2 - 3 Zimmer',
       '4 - 6 Zimmer', '6 und mehr Zimmer', '1 - 6 Zimmer',
       '2 - 6 und mehr Zimmer', '1 Zimmer', '6 Zimmer', '5 - 6 Zimmer',
       '5 - 6 und mehr Zimmer', '3 - 6 und mehr Zimmer',
       '2- bis 4-Zimmer', '2-bis 4-Zimmer', '1- bis 4-Zimmer',
       '1- bis 5-Zimmer', '1 bis 5 Zimmer', '5- bis 4-Zimmer',
       '2-6-Zimmer', '1- bis 6-Zimmer', '2- bis 5-Zimmer',
       '5- bis 5-Zimmer', '2 bis 4-Zimmer', '2-Zimmer'], dtype=object)

In [ ]:
alldata['price'] = [price.search(str(x)).group(1) if price.search(str(x)) is not None else None
                    for x in alldata.allprice]

In [ ]:
alldata2['price'] = [price.search(str(x)).group(2) if price.search(str(x)) is not None else None
                    for x in alldata2.allprice]

In [ ]:
alldata['sqft'] = [sqft.search(str(x)).group(1) if sqft.search(str(x)) is not None else None
                    for x in alldata.allsqft]

In [ ]:
alldata2['sqft'] = [sqft.search(str(x)).group(2) if sqft.search(str(x)) is not None else None
                    for x in alldata2.allsqft]

In [ ]:
alldata['rooms'] = [rooms.search(str(x)).group(1) if rooms.search(str(x)) is not None else None
                    for x in alldata.allrooms]

In [ ]:
alldata2['rooms'] = [rooms.search(str(x)).group(2) if rooms.search(str(x)) is not None else None
                    for x in alldata2.allrooms]

In [ ]:
alldata[['allprice','price','allsqft','sqft','allrooms','rooms']].loc[:10]

,allprice,price,allsqft,sqft,allrooms,rooms
0,180.000 EUR - 444.000 EUR,180.000,35 m2 - 70 m2,35,1,1
1,537.000 EUR - 597.000 EUR,537.000,127 m2,127,5,5
2,219.000 EUR,219.000,37 m2 - 154 m2,37,1,1
3,1.070.000 EUR - 2.610.000 EUR,1.070.000,"66,50 m2 - 182 m2","66,50",2,2
4,1.090.000 EUR,1.090.000,"82,50 m2","82,50",2,2
5,1.585.000 EUR,1.585.000,"134,75 m2","134,75",3,3
6,1.830.000 EUR,1.830.000,133 m2,133,3,3
7,1.940.000 EUR,1.940.000,154 m2,154,4,4
8,2.610.000 EUR,2.610.000,181 m2,181,5,5
9,181.937 EUR,181.937,37 m2 - 104 m2,37,2,2


In [ ]:
alldata2[['allprice','price','allsqft','sqft','allrooms','rooms']].loc[:10]

,allprice,price,allsqft,sqft,allrooms,rooms
0,180.000 EUR - 444.000 EUR,444.000,35 m2 - 70 m2,70,1 - 2 Zimmer,2
1,537.000 EUR - 597.000 EUR,597.000,127 m2,None,5 Zimmer,None
2,219.000 EUR,None,37 m2 - 154 m2,154,1 - 4 Zimmer,4
3,1.070.000 EUR - 2.610.000 EUR,2.610.000,"66,50 m2 - 182 m2",182,2 - 5 Zimmer,5
4,1.090.000 EUR,None,"82,50 m2",None,,None
5,1.585.000 EUR,None,"134,75 m2",None,,None
6,1.830.000 EUR,None,133 m2,None,,None
7,1.940.000 EUR,None,154 m2,None,,None
8,2.610.000 EUR,None,181 m2,None,,None
9,181.937 EUR,None,37 m2 - 104 m2,104,2 - 4 Zimmer,4


In [ ]:
alldata['unit_id']=1

In [ ]:
alldata2['unit_id']=2

In [ ]:
alldata2 = alldata2.fillna(value=np.nan)
alldata = alldata.fillna(value=np.nan)

In [ ]:
alldata2[['allprice','price','allsqft','sqft','allrooms','rooms']].loc[:10]

,allprice,price,to_drop,allsqft,sqft,allrooms,rooms
0,379.000 EUR - 759.000 EUR,759.000,False,46 m2 - 85 m2,85,2 - 3 Zimmer,3
1,219.000 EUR,NaN,True,37 m2 - 154 m2,154,1 - 4 Zimmer,4
2,233.700 EUR - 845.950 EUR,845.950,False,32 m2 - 118 m2,118,1 - 5 Zimmer,5
3,980.000 EUR - 2.345.000 EUR,2.345.000,False,"66,50 m2 - 182 m2",182,2 - 5 Zimmer,5
4,181.937 EUR,NaN,True,37 m2 - 104 m2,104,2 - 4 Zimmer,4
5,265.000 EUR - 1.239.000 EUR,1.239.000,False,36 m2 - 145 m2,145,1 - 4 Zimmer,4
6,1.295.000 EUR - 2.550.000 EUR,2.550.000,False,120 m2 - 255 m2,255,4 - 6 und mehr Zimmer,6
7,299.500 EUR - 1.099.000 EUR,1.099.000,False,"43,51 m2 - 148,65 m2","148,65",1 - 4 Zimmer,4
8,699.000 EUR,NaN,True,"132,68 m2",NaN,4 Zimmer,NaN
9,,NaN,True,"139,36 m2",NaN,5 Zimmer,NaN


In [ ]:
alldata2.dropna(subset=['price', 'sqft', 'rooms'], how='all',inplace=True)

In [ ]:
alldata=alldata.append(alldata2, ignore_index=True)

In [ ]:
len(alldata)

6598

In [ ]:
print(alldata.columns)

Index(['url', 'text', 'cleantext', 'scraped_class', 'dev_status', 'address_sc',
       'address_clean', 'is_unit', 'location', 'point', 'latitude',
       'longitude', 'altitude', 'allprice', 'allrooms', 'allsqft', 'price',
       'sqft', 'rooms', 'unit_id'],
      dtype='object')


In [ ]:
alldata.drop(['scraped_class','altitude'],axis = 1, inplace=True)

In [ ]:
alldata['price']= [re.sub("[\.]","",row) if pd.notna(row) else row for row in alldata['price']]
alldata["price"] = pd.to_numeric(alldata["price"])

In [ ]:
alldata[['allprice','price','allsqft','sqft','allrooms','rooms']].loc[20:300]

,allprice,price,allsqft,sqft,allrooms,rooms
20,414.715 EUR - 1.066.883 EUR,414715.0,"67,95 m2 - 154,45 m2","67,95",3 - 5 Zimmer,3
21,,NaN,31 m2 - 320 m2,31,1 - 6 und mehr Zimmer,1
22,189.759 EUR - 389.802 EUR,189759.0,"51,75 m2 - 91,96 m2","51,75",2 - 4 Zimmer,2
23,220.401 EUR,220401.0,"54,42 m2","54,42",2,2
24,220.401 EUR,220401.0,"54,42 m2","54,42",2,2
...,...,...,...,...,...,...
296,449.900 EUR,449900.0,"67,75 m2","67,75",2,2
297,553.400 EUR,553400.0,"87,86 m2","87,86",3,3
298,559.300 EUR,559300.0,"86,45 m2","86,45",3,3
299,745.300 EUR,745300.0,"112,92 m2","112,92",4,4


In [ ]:
alldata.allrooms.unique()

In [ ]:
alldata['sqft']= [re.sub("[\,]",".",row) if pd.notna(row) else row for row in alldata['sqft']]
alldata["sqft"] = pd.to_numeric(alldata["sqft"])

In [ ]:
wohntyp = re.compile("Wohntyp ([\S]+)")

In [ ]:
alldata['wohntyp'] = [wohntyp.search(str(x)).group(1) if wohntyp.search(str(x)) is not None else None
                    for x in alldata.cleantext]

In [ ]:
alldata.wohntyp.unique()

array(['Eigentumswohnung', 'Doppelhaushalfte', 'Apartment',
       'Etagenwohnung', 'Dachgeschosswohnung', 'Erdgeschosswohnung',
       'Reihenhaus', None, 'Penthouse', 'Maisonettewohnung',
       'Terrassenwohnung', 'Einfamilienhaus', 'Souterrainwohnung',
       'Reihenendhaus', 'Loft', 'Reihenmittelhaus', 'Galeriewohnung',
       'Reiheneckhaus', 'Bungalow', 'Stadthaus', 'Townhouses'],
      dtype=object)

In [ ]:
alldata.to_csv('alldata.csv')